In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import string
import time

import sklearn
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,ExtraTreesClassifier
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import f1_score

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import spacy

In [20]:
data_audio = pd.read_csv('audio_features.csv')
#data_audio = data_audio.loc[data_audio['2'] == 'Monica']
data_audio  = np.asarray(data_audio)

In [21]:
df1 = pd.read_csv('data1.csv')
df2 = pd.read_csv('data2.csv')
df3 = pd.read_csv('data3.csv')
df4 = pd.read_csv('data4.csv')

frames = [df1,df2,df3,df4]
data_text = pd.concat(frames) #complete dataset
#data_text = data_text.loc[data_text['2'] == 'Monica']
mat =np.asarray([]) #emo
mat1 =np.asarray([]) #senti
data_text = np.asarray(data_text[['1','3','4']])

In [22]:
data_audio[:,4:6]

array([[0, 0],
       [2, 2],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]], dtype=object)

In [23]:
data_text[:,1:3]

array([[0, 0],
       [2, 2],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]], dtype=object)

In [24]:
print(np.array_equal(data_text[:,1:3], data_audio[:,4:6]))

True


In [25]:
nlp = spacy.load('en_core_web_lg')

total_vectors = len(nlp.vocab.vectors)
print('Total word vectors:', total_vectors)
doc_glove_vectors = np.array([nlp(str(doc)).vector for doc in data_text[:,0]])
X_glove = np.zeros((doc_glove_vectors.shape[0], 300))
for i in range(doc_glove_vectors.shape[0]):
    if (doc_glove_vectors[i].shape[0] == 300):
        X_glove[i,:] = doc_glove_vectors[i][:]
    else:
        print(i)


Total word vectors: 684830


In [26]:
scaler = MinMaxScaler()
feat_combined = np.concatenate((scaler.fit_transform(X_glove), scaler.fit_transform(data_audio[:,13:])), axis = 1)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(feat_combined, data_audio[:,2:6], test_size=0.2, shuffle = True)

y_train = y_train[y_train[:,1] == "Rachel"]

# emotion (7)
y_train_emo = y_train[:,2] *1.0
y_train_emo = y_train_emo.astype('float')
y_test_emo = y_test[:,2] *1.0
y_test_emo = y_test_emo.astype('float')

# sentiment(3)
y_train_senti = y_train[:,3] *1.0
y_train_senti = y_train_senti.astype('float')
y_test_senti = y_test[:,3] *1.0
y_test_senti = y_test_senti.astype('float')

In [ ]:
def logistic_regression(X_train, y_train, X_test, y_test):
    print("Multinomial Logistic Regression")
    start = time.time()
    logreg = LogisticRegression(penalty='l1',multi_class='multinomial', max_iter = 1e4, solver = 'saga')
    #penalty to handle size better
    
    #https://towardsdatascience.com/dont-sweat-the-solver-stuff-aea7cddc3451
    
    logreg.fit(X_train, y_train)
    accuracy_train_data = cross_val_score(logreg, X_train,y_train,cv=5)
    print("Training Accuracy : ",np.mean(accuracy_train_data))
    accuracy_test_data = cross_val_score(logreg, X_test,y_test,cv=5)
    print("Testing Accuracy : ",np.mean(accuracy_test_data))
    y_pred = logreg.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    print("Time Taken :", (time.time()-start))
    return np.mean(accuracy_train_data), np.mean(accuracy_test_data), time.time()-start

print("FOR EMOTIONS (7): ")
tup = logistic_regression(X_train, y_train_emo, X_test, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = logistic_regression(X_train, y_train_senti, X_test, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Multinomial Logistic Regression


In [11]:
def random_forest(X_train, y_train, X_test, y_test):
    start = time.time()
    print("Random Forest Classifier")
    rf = RandomForestClassifier(random_state=42)
    rf.fit(X_train, y_train)
    accuracy_train_data = cross_val_score(rf, X_train,y_train,cv=5)
    print("Training Accuracy : ",np.mean(accuracy_train_data))

    # predicting test set results
    y_pred = rf.predict(X_test)
    accuracy_test_data = cross_val_score(rf, X_test,y_test,cv=5)
    print("Testing Accuracy : ",np.mean(accuracy_test_data))
    # making the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    f1 = f1_score(y_test, y_pred, average='micro')
    print("F1-Score : ",f1)
    print("Time Taken :", (time.time()-start))
    return np.mean(accuracy_train_data), np.std(accuracy_train_data), np.mean(accuracy_test_data), np.std(accuracy_test_data), f1, time.time()-start


print("FOR EMOTIONS (7): ")
tup = random_forest(X_train, y_train_emo, X_test, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup= random_forest(X_train, y_train_senti, X_test, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Random Forest Classifier
Training Accuracy :  0.5538027061325898
Testing Accuracy :  0.5269853615731324
[[1240   17    0    0   24    0    3]
 [ 189   79    0    0   43    0   17]
 [  56    3    0    0    7    0    6]
 [ 169    4    0    0   13    0    7]
 [ 301    9    0    0  150    0   15]
 [  54    4    0    0    5    0    8]
 [ 197    9    0    0   59    0   54]]
F1-Score :  0.5554339897884756
Time Taken : 162.17528986930847

FOR SENTIMENTS (3): 
Random Forest Classifier
Training Accuracy :  0.6116174735141773
Testing Accuracy :  0.5878990334117772
[[1180   18   86]
 [ 299  181  155]
 [ 429   79  315]]
F1-Score :  0.611232676878191
Time Taken : 139.73934364318848


In [ ]:
def sv_classifier(X_train, y_train, X_test, y_test):
    start = time.time()
    print("Support Vector Classifier")
    clf = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto'))
    clf.fit(X_train, y_train)
    accuracy_train_data = cross_val_score(clf, X_train,y_train,cv=5)
    print("Training Accuracy : ",np.mean(accuracy_train_data))
    
    # predicting test set results
    y_pred = clf.predict(X_test)
    accuracy_test_data = cross_val_score(clf, X_test,y_test,cv=5)
    print("Testing Accuracy : ",np.mean(accuracy_test_data))
    # making the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='micro')
    print("F1-Score : ",f1)
    print(cm)
    print("Time Taken :", (time.time()-start))
    return np.mean(accuracy_train_data), np.std(accuracy_train_data), np.mean(accuracy_test_data), np.std(accuracy_test_data), f1, time.time()-start

print("FOR EMOTIONS (7): ")
tup = sv_classifier(X_train, y_train_emo, X_test, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = sv_classifier(X_train, y_train_senti, X_test, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

In [28]:
from sklearn.decomposition import PCA

In [29]:
pca = PCA(n_components=300, svd_solver='full')
X_pca = pca.fit_transform(data_audio[:,13:].copy())
print(sum(pca.explained_variance_ratio_))

0.8449567202155046


In [30]:
feat_combined = np.concatenate((scaler.fit_transform(X_glove), scaler.fit_transform(X_pca)), axis = 1)

In [33]:
print(feat_combined.shape)

(13708, 600)


In [34]:
X_train, X_test, y_train, y_test = train_test_split(feat_combined, data_audio[:,2:6], test_size=0.2, shuffle = True)
# emotion (7)
# X_test = X_test[y_test[:,1] == "Joey"]
# y_test = y_test[y_test[:,1] == "Joey"]
# #args = y_test[:,1] == "Chandler"

# y_train_emo = y_train[:,2] *1.0
# y_train_emo = y_train_emo.astype('float')
# y_test_emo = y_test[:,2] *1.0
# y_test_emo = y_test_emo.astype('float')

# # sentiment(3)
# y_train_senti = y_train[:,3] *1.0
# y_train_senti = y_train_senti.astype('float')
# y_test_senti = y_test[:,3] *1.0
# y_test_senti = y_test_senti.astype('float')

In [16]:
X_testi = X_test.copy()
y_testi = y_test.copy()

In [35]:
#y_train = y_train[y_train[:,1] == "Chandler"]
# X_test = X_testi[y_testi[:,1] == "Chandler"]
# y_test = y_testi[y_testi[:,1] == "Chandler"]
# #args = y_test[:,1] == "Chandler"
# print(X_test.shape, y_test.shape)

y_train_emo = y_train[:,2] *1.0
y_train_emo = y_train_emo.astype('float')
y_test_emo = y_test[:,2] *1.0
y_test_emo = y_test_emo.astype('float')

# sentiment(3)
y_train_senti = y_train[:,3] *1.0
y_train_senti = y_train_senti.astype('float')
y_test_senti = y_test[:,3] *1.0
y_test_senti = y_test_senti.astype('float')

In [107]:
def sv_classifier(X_train, y_train, X_test, y_test):
    start = time.time()
    print("Support Vector Classifier")
    clf = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto'))
    clf.fit(X_train, y_train)
    accuracy_train_data = cross_val_score(clf, X_train,y_train,cv=5)
    print("Training Accuracy : ",np.mean(accuracy_train_data))
    
    # predicting test set results
    y_pred = clf.predict(X_test)
    accuracy_test_data = cross_val_score(clf, X_test,y_test,cv=5)
    print("Testing Accuracy : ",np.mean(accuracy_test_data))
    # making the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='micro')
    print("F1-Score : ",f1)
    print(cm)
    print("Time Taken :", (time.time()-start))
    return np.mean(accuracy_train_data), np.std(accuracy_train_data), np.mean(accuracy_test_data), np.std(accuracy_test_data), f1, time.time()-start

print("FOR EMOTIONS (7): ")
tup = sv_classifier(X_train, y_train_emo, X_test, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = sv_classifier(X_train, y_train_senti, X_test, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Support Vector Classifier
Training Accuracy :  0.598213051305617


C:\Users\saisi\anaconda3\envs\sidenv\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\saisi\anaconda3\envs\sidenv\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\saisi\anaconda3\envs\sidenv\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\saisi\anaconda3\envs\sidenv\lib\site-packages\sklearn\svm\_base.py", line 199, in fit
    y = self._validate_

Testing Accuracy :  nan
F1-Score :  0.9291217257318953
[[1206   22    1    3   49   17]
 [   0    0    0    0    0    0]
 [   0    0    0    0    0    0]
 [   0    0    0    0    0    0]
 [   0    0    0    0    0    0]
 [   0    0    0    0    0    0]]
Time Taken : 196.60517024993896

FOR SENTIMENTS (3): 
Support Vector Classifier
Training Accuracy :  0.6556633541462206
Testing Accuracy :  nan
F1-Score :  0.884437596302003
[[1148   48  102]
 [   0    0    0]
 [   0    0    0]]
Time Taken : 170.23407292366028


C:\Users\saisi\anaconda3\envs\sidenv\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\saisi\anaconda3\envs\sidenv\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\saisi\anaconda3\envs\sidenv\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\saisi\anaconda3\envs\sidenv\lib\site-packages\sklearn\svm\_base.py", line 199, in fit
    y = self._validate_

In [24]:
def sv_classifier(X_train, y_train, X_test, y_test):
    start = time.time()
    print("Support Vector Classifier")
    clf = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto'))
    clf.fit(X_train, y_train)
    accuracy_train_data = cross_val_score(clf, X_train,y_train,cv=5)
    print("Training Accuracy : ",np.mean(accuracy_train_data))
    
    # predicting test set results
    y_pred = clf.predict(X_test)
    accuracy_test_data = cross_val_score(clf, X_test,y_test,cv=5)
    print("Testing Accuracy : ",np.mean(accuracy_test_data))
    # making the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='micro')
    print("F1-Score : ",f1)
    print(cm)
    print("Time Taken :", (time.time()-start))
    return np.mean(accuracy_train_data), np.std(accuracy_train_data), np.mean(accuracy_test_data), np.std(accuracy_test_data), f1, time.time()-start

print("FOR EMOTIONS (7): ")
tup = sv_classifier(X_train, y_train_emo, X_test, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = sv_classifier(X_train, y_train_senti, X_test, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Support Vector Classifier
Training Accuracy :  0.597575570899535
Testing Accuracy :  0.5692938720699879
F1-Score :  0.6032093362509118
[[1228   19    0    2   51    0   12]
 [ 136  111    0    0   63    0   24]
 [  42    3    0    2    6    0   13]
 [ 170   10    0    9   11    0   12]
 [ 171   16    1    1  207    0   39]
 [  45    6    0    1    5    0   11]
 [ 140   21    0    1   54    0   99]]
Time Taken : 250.80271935462952

FOR SENTIMENTS (3): 
Support Vector Classifier
Training Accuracy :  0.6546614923343148
Testing Accuracy :  0.6404059138712722
F1-Score :  0.675054704595186
[[1162   52   98]
 [ 192  281  133]
 [ 345   71  408]]
Time Taken : 228.58726835250854


In [43]:
Y_train_senti = np.zeros((y_train_senti.shape[0], 3))
for i in range(y_train_senti.shape[0]):
    if y_train_senti[i] == 0:
        Y_train_senti[i,:] = [1, 0, 0]
    if y_train_senti[i] == 1:
        Y_train_senti[i,:] = [0, 1, 0]
    if y_train_senti[i] == 2:
        Y_train_senti[i,:] = [0, 0, 1]

In [44]:
Y_test_senti = np.zeros((y_test_senti.shape[0], 3))
for i in range(y_test_senti.shape[0]):
    if y_test_senti[i] == 0:
        Y_test_senti[i,:] = [1, 0, 0]
    if y_test_senti[i] == 1:
        Y_test_senti[i,:] = [0, 1, 0]
    if y_test_senti[i] == 2:
        Y_test_senti[i,:] = [0, 0, 1]

In [62]:
import numpy as np
import scipy.io
import torch
import torchvision
from torch import nn, optim
import torch.nn.functional as F  # a lower level (compared to torch.nn) interface
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from time import time
from torch.utils.data import TensorDataset, DataLoader


torch.manual_seed(123)
device = torch.device("cpu")

train_x, train_y = X_train.astype(np.float32), Y_train_senti.astype(np.int)
valid_x, valid_y = X_test.astype(np.float32), Y_test_senti.astype(np.int)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(600, 3000)
        self.fc2 = nn.Linear(3000, 2000)
        self.fc3 = nn.Linear(2000, 1000)
        self.fc4 = nn.Linear(1000, 500)
        self.fc5 = nn.Linear(500, 300)
        self.fc6 = nn.Linear(300, 3)
#         self.fc5 = nn.Linear(2000, 1000)
#         self.fc6 = nn.Linear(1000, 500)
#         self.fc7 = nn.Linear(500, 3)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc3(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc4(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc5(x))
        x = F.dropout(x, training=self.training)
        x = self.fc6(x)
        return x
        

model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.0005, momentum=0.9)

givendata_train = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
givendata_test = TensorDataset(torch.from_numpy(valid_x), torch.from_numpy(valid_y))
trainset_loader = DataLoader(givendata_train, batch_size=64, shuffle=True, num_workers=1)
validset_loader = DataLoader(givendata_test, batch_size=64, shuffle=True, num_workers=1)


def train(max_iters):
    model.train()
    Taccuracies = []
    Tlosses = []
    for itr in range(max_iters):
        correct = 0
        Tloss = 0
        num = 0
        for batch_idx, (data, target) in enumerate(trainset_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            target = torch.max(target, 1)[1]
            loss = nn.functional.cross_entropy(output, target)            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            Tloss = Tloss + loss.item()

            pred = torch.max(output, 1)[1] 
            correct += pred.eq(target).sum().item()
            num = num + 1
        
        Taccuracies.append(100. * correct / (num*64))
        Tlosses.append(Tloss/num)
        if itr % 10 == 0:
            print('Accuracy {:.2f} %'.format(100. * correct / (num*64)))
            print('Loss: {:.6f}'.format(Tloss/num))
        
        with torch.no_grad():
            correct = 0
            Tloss = 0
            num = 0
            for batch_idx, (data, target) in enumerate(validset_loader):
                data, target = data.to(device), target.to(device)
                output = model(data)
                target = torch.max(target, 1)[1]
                loss = nn.functional.cross_entropy(output, target)            
                Tloss = Tloss + loss.item()

                pred = torch.max(output, 1)[1] 
                correct += pred.eq(target).sum().item()
                num = num + 1

            Taccuracies.append(100. * correct / (num*64))
            Tlosses.append(Tloss/num)
            if itr % 10 == 0:
                print('Test Accuracy {:.2f} %'.format(100. * correct / (num*64)))
                print('Test Loss: {:.6f}'.format(Tloss/num))

    return Taccuracies, Tlosses

C:\Users\saisi\anaconda3\envs\sidenv\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
C:\Users\saisi\anaconda3\envs\sidenv\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for 

In [64]:
Taccuracies, Tlosses = train(100)

Accuracy 56.50 %
Loss: 0.907798
Test Accuracy 57.09 %
Test Loss: 0.906664
Accuracy 57.85 %
Loss: 0.884336
Test Accuracy 58.21 %
Test Loss: 0.900535
Accuracy 58.36 %
Loss: 0.874906
Test Accuracy 58.18 %
Test Loss: 0.886666
Accuracy 58.74 %
Loss: 0.868387
Test Accuracy 59.12 %
Test Loss: 0.879699
Accuracy 59.52 %
Loss: 0.865622
Test Accuracy 59.67 %
Test Loss: 0.882686
Accuracy 59.27 %
Loss: 0.854918
Test Accuracy 59.81 %
Test Loss: 0.870758
Accuracy 59.97 %
Loss: 0.847769
Test Accuracy 60.17 %
Test Loss: 0.865348
Accuracy 60.14 %
Loss: 0.838698
Test Accuracy 58.39 %
Test Loss: 0.875197
Accuracy 60.14 %
Loss: 0.842262
Test Accuracy 59.48 %
Test Loss: 0.869486
Accuracy 60.43 %
Loss: 0.834213
Test Accuracy 59.08 %
Test Loss: 0.872016


In [65]:
Taccuracies, Tlosses = train(100)

Accuracy 60.94 %
Loss: 0.821893
Test Accuracy 60.03 %
Test Loss: 0.869604
Accuracy 61.56 %
Loss: 0.809883
Test Accuracy 59.01 %
Test Loss: 0.905895
Accuracy 62.92 %
Loss: 0.789188
Test Accuracy 59.63 %
Test Loss: 0.877293
Accuracy 63.00 %
Loss: 0.783700
Test Accuracy 57.09 %
Test Loss: 0.893886
Accuracy 63.69 %
Loss: 0.758338
Test Accuracy 56.58 %
Test Loss: 0.904735
Accuracy 65.72 %
Loss: 0.731071
Test Accuracy 57.70 %
Test Loss: 0.907084
Accuracy 65.27 %
Loss: 0.725349
Test Accuracy 59.27 %
Test Loss: 0.951403
Accuracy 67.10 %
Loss: 0.678583
Test Accuracy 58.83 %
Test Loss: 0.952845
Accuracy 67.51 %
Loss: 0.683906
Test Accuracy 56.40 %
Test Loss: 0.924327
Accuracy 67.55 %
Loss: 0.665514
Test Accuracy 58.21 %
Test Loss: 1.032440


In [66]:
Taccuracies, Tlosses = train(100)

Accuracy 70.03 %
Loss: 0.608035
Test Accuracy 54.03 %
Test Loss: 1.031826
Accuracy 67.43 %
Loss: 0.676776
Test Accuracy 53.20 %
Test Loss: 0.986071
Accuracy 71.17 %
Loss: 0.602814
Test Accuracy 50.80 %
Test Loss: 1.149753


KeyboardInterrupt: 

In [36]:
Y_train_emo = np.zeros((y_train_emo.shape[0], 7))
for i in range(y_train_emo.shape[0]):
    if y_train_emo[i] == 0:
        Y_train_emo[i,:] = [1, 0, 0, 0, 0, 0, 0]
    if y_train_emo[i] == 1:
        Y_train_emo[i,:] = [0, 1, 0, 0, 0, 0, 0]
    if y_train_emo[i] == 2:
        Y_train_emo[i,:] = [0, 0, 1, 0, 0, 0, 0]
    if y_train_emo[i] == 3:
        Y_train_emo[i,:] = [0, 0, 0, 1, 0, 0, 0]
    if y_train_emo[i] == 4:
        Y_train_emo[i,:] = [0, 0, 0, 0, 1, 0, 0]
    if y_train_emo[i] == 5:
        Y_train_emo[i,:] = [0, 0, 0, 0, 0, 1, 0]
    if y_train_emo[i] == 6:
        Y_train_emo[i,:] = [0, 0, 0, 0, 0, 0, 1]
        
        
Y_test_emo = np.zeros((y_test_emo.shape[0], 7))
for i in range(y_test_emo.shape[0]):
    if y_test_emo[i] == 0:
        Y_test_emo[i,:] = [1, 0, 0, 0, 0, 0, 0]
    if y_test_emo[i] == 1:
        Y_test_emo[i,:] = [0, 1, 0, 0, 0, 0, 0]
    if y_test_emo[i] == 2:
        Y_test_emo[i,:] = [0, 0, 1, 0, 0, 0, 0]
    if y_test_emo[i] == 3:
        Y_test_emo[i,:] = [0, 0, 0, 1, 0, 0, 0]
    if y_test_emo[i] == 4:
        Y_test_emo[i,:] = [0, 0, 0, 0, 1, 0, 0]
    if y_test_emo[i] == 5:
        Y_test_emo[i,:] = [0, 0, 0, 0, 0, 1, 0]
    if y_test_emo[i] == 6:
        Y_test_emo[i,:] = [0, 0, 0, 0, 0, 0, 1]

In [37]:
import numpy as np
import scipy.io
import torch
import torchvision
from torch import nn, optim
import torch.nn.functional as F  # a lower level (compared to torch.nn) interface
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from time import time
from torch.utils.data import TensorDataset, DataLoader


torch.manual_seed(123)
device = torch.device("cpu")

train_x, train_y = X_train.astype(np.float32), Y_train_emo.astype(np.int)
valid_x, valid_y = X_test.astype(np.float32), Y_test_emo.astype(np.int)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(600, 3000)
        self.fc2 = nn.Linear(3000, 2000)
        self.fc3 = nn.Linear(2000, 1000)
        self.fc4 = nn.Linear(1000, 500)
        self.fc5 = nn.Linear(500, 300)
        self.fc6 = nn.Linear(300, 7)
#         self.fc5 = nn.Linear(2000, 1000)
#         self.fc6 = nn.Linear(1000, 500)
#         self.fc7 = nn.Linear(500, 3)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc3(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc4(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc5(x))
        x = F.dropout(x, training=self.training)
        x = self.fc6(x)
        return x
        

model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.0005, momentum=0.9)

givendata_train = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
givendata_test = TensorDataset(torch.from_numpy(valid_x), torch.from_numpy(valid_y))
trainset_loader = DataLoader(givendata_train, batch_size=64, shuffle=True, num_workers=1)
validset_loader = DataLoader(givendata_test, batch_size=64, shuffle=True, num_workers=1)


def train(max_iters):
    model.train()
    Taccuracies = []
    Tlosses = []
    for itr in range(max_iters):
        correct = 0
        Tloss = 0
        num = 0
        for batch_idx, (data, target) in enumerate(trainset_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            target = torch.max(target, 1)[1]
            loss = nn.functional.cross_entropy(output, target)            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            Tloss = Tloss + loss.item()

            pred = torch.max(output, 1)[1] 
            correct += pred.eq(target).sum().item()
            num = num + 1
        
        Taccuracies.append(100. * correct / (num*64))
        Tlosses.append(Tloss/num)
        if itr % 10 == 0:
            print('Accuracy {:.2f} %'.format(100. * correct / (num*64)))
            print('Loss: {:.6f}'.format(Tloss/num))
        
        with torch.no_grad():
            correct = 0
            Tloss = 0
            num = 0
            for batch_idx, (data, target) in enumerate(validset_loader):
                data, target = data.to(device), target.to(device)
                output = model(data)
                target = torch.max(target, 1)[1]
                loss = nn.functional.cross_entropy(output, target)            
                Tloss = Tloss + loss.item()

                pred = torch.max(output, 1)[1] 
                correct += pred.eq(target).sum().item()
                num = num + 1

            Taccuracies.append(100. * correct / (num*64))
            Tlosses.append(Tloss/num)
            if itr % 10 == 0:
                print('Test Accuracy {:.2f} %'.format(100. * correct / (num*64)))
                print('Test Loss: {:.6f}'.format(Tloss/num))

    return Taccuracies, Tlosses

C:\Users\saisi\anaconda3\envs\sidenv\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
C:\Users\saisi\anaconda3\envs\sidenv\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for 

In [38]:
Taccuracies, Tlosses = train(250)

Accuracy 39.05 %
Loss: 1.887980
Test Accuracy 46.33 %
Test Loss: 1.815119
Accuracy 46.88 %
Loss: 1.549549
Test Accuracy 46.33 %
Test Loss: 1.548742
Accuracy 46.88 %
Loss: 1.545137
Test Accuracy 46.33 %
Test Loss: 1.544692
Accuracy 46.88 %
Loss: 1.544661
Test Accuracy 46.33 %
Test Loss: 1.545851
Accuracy 46.88 %
Loss: 1.540194
Test Accuracy 46.33 %
Test Loss: 1.538619
Accuracy 46.88 %
Loss: 1.531877
Test Accuracy 46.33 %
Test Loss: 1.530019
Accuracy 46.88 %
Loss: 1.487429
Test Accuracy 46.33 %
Test Loss: 1.487566
Accuracy 47.14 %
Loss: 1.422493
Test Accuracy 47.09 %
Test Loss: 1.428863
Accuracy 52.55 %
Loss: 1.378669
Test Accuracy 51.82 %
Test Loss: 1.421778
Accuracy 53.01 %
Loss: 1.363971
Test Accuracy 52.87 %
Test Loss: 1.396018
Accuracy 53.05 %
Loss: 1.354155
Test Accuracy 53.05 %
Test Loss: 1.392738
Accuracy 53.18 %
Loss: 1.338556
Test Accuracy 52.94 %
Test Loss: 1.376449
Accuracy 53.16 %
Loss: 1.343432
Test Accuracy 52.73 %
Test Loss: 1.364215
Accuracy 53.33 %
Loss: 1.323867
Test A